First install this [reddit picture scraper](https://github.com/impshum/Multithreaded-Reddit-Image-Downloader)


In [4]:
!pip3 install -r requirements.txt
!wget https://raw.githubusercontent.com/JohnnyUrosevic/Multithreaded-Reddit-Image-Downloader/master/run.py

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement tensorflow==1.14 (from -r requirements.txt (line 6)) (from versions: 2.2.0rc1, 2.2.0rc2, 2.2.0rc3, 2.2.0rc4, 2.2.0, 2.3.0rc0, 2.3.0rc1, 2.3.0rc2, 2.3.0)
ERROR: No matching distribution found for tensorflow==1.14 (from -r requirements.txt (line 6))
--2020-09-16 17:41:56--  https://raw.githubusercontent.com/JohnnyUrosevic/Multithreaded-Reddit-Image-Downloader/master/run.py
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.196.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.196.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2675 (2.6K) [text/plain]
Saving to: ‘run.py.1’

run.py.1            100%[===================>]   2.61K  --.-KB/s    in 0s      

2020-09-16 17:41:56 (5.17 MB/s) - ‘run.py.1’ saved [2675

Here we set the parameters for testing

In [14]:
image_count = 10
safe_paths = []
nsfw_paths = []

safe_subreddits = ["pics", "gaming"]
nsfw_subreddits = ["nsfw", "rule34"]

In [15]:
from run import redditImageScraper

def get_images(subreddit, nsfw):
    destination = nsfw_paths if nsfw else safe_paths
    destination.append(f"images/{subreddit}")
    
    scraper = redditImageScraper(subreddit, image_count, "hot", nsfw)
    scraper.start()

for sub in safe_subreddits:
    get_images(sub, False)
for sub in nsfw_subreddits:
    get_images(sub, True)

In [2]:
from nudenet import NudeClassifier
classifier = NudeClassifier()

Using TensorFlow backend.
/home/goose/Code/discord-nudity-bot/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/goose/Code/discord-nudity-bot/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/goose/Code/discord-nudity-bot/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/h









Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [34]:
import numpy as np
confusion = np.zeros((2,2))
print(confusion)

[[0. 0.]
 [0. 0.]]


In [35]:
# Test safe images
import os

def test(paths, nsfw, threshold):
    for path in paths:
        prob = classifier.classify([os.path.join(path,f) for f in os.listdir(path)])
        for p in prob.values():
            if p['unsafe'] >= threshold:
                confusion[nsfw][1] += 1
            else:
                confusion[nsfw][0] += 1

test(safe_paths, 0, .9)
test(nsfw_paths, 1, .9)
print(confusion)

[[40.  0.]
 [ 8. 32.]]


[[40.  0.]
 [ 8. 32.]]
